In [3]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("BR|FSS2023_1000.csv")

OSError: [Errno 22] Invalid argument: 'BR|FSS2023_1000.csv'

In [7]:
df.head()

NameError: name 'df' is not defined

In [37]:
#df.columns.tolist()

In [1]:
cols = ['cimemlo1', 'avedrnk3', '_smoker3', 'exeroft1', '_rfbmi5', 'genhlth']
# cimemlo1: current memory loss
# avedrnk3: average number of drinks per day
# _smoker3: current smoking status
# exeroft1: exercise frequency
# _rfbmi5: BMI category
# genhlth: general health status


In [2]:
df[cols].dropna().value_counts()


NameError: name 'df' is not defined